In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


### Imports (run all)

In [ ]:
import os
os.chdir('drive/MyDrive/stellargraph') # edited stellargraph requirments to work with python 3.9
%pip install .
os.chdir('..')

In [ ]:
!pip install kaleido 

In [ ]:
import kaleido
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import json
from tqdm.auto import tqdm
import ast
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import random
import plotly.graph_objects as go
import pickle
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Conv1D, MaxPooling1D, Flatten
import networkx as nx
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE, MeanAggregator, MaxPoolingAggregator, MeanPoolingAggregator, AttentionalAggregator
from stellargraph import StellarGraph
import stellargraph as sg
from tensorflow.keras import layers, optimizers, losses, metrics, Model, initializers
import tensorflow as tf
import tensorflow.keras.backend as K
import warnings
from math import radians, sin, cos, sqrt, atan2
import gc
import itertools

In [ ]:
def nice_plot(fig,  x_label=None, y_label=None, title=None, height=550, width=800, legend=True, y_range=None, x_range=None):
    # set background to white
    fig.update_layout(plot_bgcolor='white')
    # change fig size
    fig.update_layout(height=height, width=width)
    # change x axis title
    if x_label:
        fig.update_xaxes(title_text=x_label)
    #change y axis title
    if y_label:
        fig.update_yaxes(title_text=y_label)

    # change title
    if title:
        fig.update_layout(title_text=title, title_x=0.5)

    if not legend:
        fig.update_layout(showlegend=False)

    if y_range:
        fig.update_layout(yaxis_range=y_range)
        
    # fig.update_layout(
    #     margin=dict(l=0, r=0, b=0, t=0),
    # )
    
    # change font to latex font
    fig.update_layout(font_family='serif')
    fig.update_layout(font_size=20)
    
    # make font color black
    fig.update_layout(font_color='black')
    
    # add axis lines
    fig.update_yaxes(showline=True,  # add line at x=0
                     linecolor='black',  # line color
                     linewidth=2.4,  # line size
                     #  ticks='outside',  # ticks outside axis
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=2.4,  # tick width
                     tickcolor='black',  # tick color
                     )
    fig.update_xaxes(showline=True,
                     showticklabels=True,
                     linecolor='black',
                     linewidth=2.4,
                     #  ticks='outside',
                     mirror='allticks',
                     tickwidth=2.4,
                     tickcolor='black',
                     )
    return fig


In [ ]:
bristol = pd.read_feather('bristol_ground_sat_weather 2.feather')

with open('lat_long_to_location.pkl', 'rb') as f:
    lat_long_to_location = pickle.load(f)
bristol.lat_long = bristol.lat_long.apply(lambda x: tuple(x))

In [ ]:
london = pd.read_feather(path='london_ground_sat_weather.feather')
london.lat_long = london.lat_long.apply(lambda x: tuple(x))

### pick df

In [ ]:
df = bristol.copy()

In [ ]:
# df = london.copy()

### normalise data

In [ ]:
df['absorbing_aerosol_index'] = (df['absorbing_aerosol_index'] - df['absorbing_aerosol_index'].mean()) / df['absorbing_aerosol_index'].std()
df['tropospheric_NO2_column_number_density'] = (df['tropospheric_NO2_column_number_density'] - df['tropospheric_NO2_column_number_density'].mean()) / df['tropospheric_NO2_column_number_density'].std()
df['temperature_2m'] = (df['temperature_2m'] - df['temperature_2m'].mean()) / df['temperature_2m'].std()
df['relativehumidity_2m'] = (df['relativehumidity_2m'] - df['relativehumidity_2m'].mean()) / df['relativehumidity_2m'].std()
df['windspeed_10m'] = (df['windspeed_10m'] - df['windspeed_10m'].mean()) / df['windspeed_10m'].std()
df['distance_to_road'] = (df['distance_to_road'] - df['distance_to_road'].mean()) / df['distance_to_road'].std()
df['dewpoint_2m'] = (df['dewpoint_2m'] - df['dewpoint_2m'].mean()) / df['dewpoint_2m'].std()
df['surface_pressure'] = (df['surface_pressure'] - df['surface_pressure'].mean()) / df['surface_pressure'].std()
df['cloudcover_low'] = (df['cloudcover_low'] - df['cloudcover_low'].mean()) / df['cloudcover_low'].std()
df['winddirection_10m'] = (df['winddirection_10m'] - df['winddirection_10m'].mean()) / df['winddirection_10m'].std()
df['windgusts_10m'] = (df['windgusts_10m'] - df['windgusts_10m'].mean()) / df['windgusts_10m'].std()
df['vapor_pressure_deficit'] = (df['vapor_pressure_deficit'] - df['vapor_pressure_deficit'].mean()) / df['vapor_pressure_deficit'].std()
df['time_of_day'] = df.time.dt.hour / 24 + df.time.dt.minute / (24 * 60) + df.time.dt.second / (24 * 60 * 60)
df['lat_long'] = df.lat_long.apply(lambda x: tuple(x))
df['day_of_week'] = df.time.dt.day_name()
df['day'] = df.time.dt.dayofyear /366
df['week'] = df.time.dt.week /53
df['lat'] = df.lat_long.apply(lambda x: x[0]).astype(float)
df['long'] = df.lat_long.apply(lambda x: x[1]).astype(float)
df['lat'] = (df['lat'] - df['lat'].mean()) / df['lat'].std()
df['long'] = (df['long'] - df['long'].mean()) / df['long'].std()
# ONE HOT ENCODE DAT BITCH
df = pd.concat([df, pd.get_dummies(df['day_of_week'])], axis=1)

# Sort the dataframe by time
df = df.sort_values('time')

# Create a new column that contains the NO2 value for that lat_long at the previous timestep
df['prev_no2'] = df.groupby('lat_long')['NO2'].shift(1)
df['prev_no2'] = df.groupby(['day', 'week', 'time_of_day'])['prev_no2'].transform(lambda x: x.fillna(x.mean()))

### define features

In [ ]:
features = ['day', 'week',
            'tropospheric_NO2_column_number_density', 'absorbing_aerosol_index',
            'temperature_2m', 'relativehumidity_2m', 'windspeed_10m', 'dewpoint_2m',
            'surface_pressure', 'cloudcover_low', 'winddirection_10m',
            'windgusts_10m', 'vapor_pressure_deficit',
            'distance_to_road', 'time_of_day', 'lat', 'long', 'Friday', 'Monday',
            'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'prev_no2'
            ]
target = 'NO2'


In [ ]:
def distance(lat_long1, lat_long2):
    # Convert to radians
    lat1, lon1 = lat_long1
    lat2, lon2 = lat_long2
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = 6371000 * c  # Radius of Earth in meters
    return distance

def get_tuple(string):
    start_index = string.index("(") + 1
    end_index = string.index(")")
    result = string[start_index:end_index]
    return tuple(map(float, result.split(",")))

In [ ]:
df['features'] = df[features].values.tolist()

# remove marlborough streeet from df as data is weird
df = df[df.lat_long != (51.459142, -2.595433)]
lat_longs = list(df.lat_long.unique())

### Define graph structure

In [ ]:
timesteps = list(df.time.unique())

def get_graph(timestep, n):
    df_timestep = df[df.time == timestep]
    lat_longs = list(df_timestep.lat_long.unique())
    G = nx.Graph()
    # create node names, string of lat long, with time step added on
    nodes = [str(lat_long) +'-' + str(n) for lat_long in lat_longs]
    G.add_nodes_from(nodes)
    
    for node, other_node in itertools.combinations(nodes, 2):
        dis = distance(get_tuple(node), get_tuple(other_node))
        if dis < 3900:  
            G.add_edge(node, other_node, weight=dis)
                
    return G

def get_graphs(timesteps):
    return [get_graph(timestep, n) for n, timestep in enumerate(tqdm(timesteps))]

graphs = get_graphs(timesteps)


# add in node features and NO2
df_dict = {}
for _, row in df.iterrows():
    time, lat_long, NO2, features = row['time'], row['lat_long'], row['NO2'], row['features']
    if time not in df_dict:
        df_dict[time] = {}
    df_dict[time][lat_long] = {'NO2': NO2, 'features': features}

for timestep, graph in tqdm(zip(timesteps, graphs)):
    timestep_dict = df_dict.get(timestep, {})
    for node in graph.nodes:
        node_tuple = get_tuple(node)
        node_dict = timestep_dict.get(node_tuple, {})
        graph.nodes[node]['NO2'] = node_dict.get('NO2', None)
        graph.nodes[node]['lat_long'] = node_tuple
        graph.nodes[node]['features'] = node_dict.get('features', None)

# create dict of node ids and features

all_nodes = {}
all_nodes_targets = {}
for graph in tqdm(graphs):
    for node in graph.nodes():
        all_nodes[node] = graph.nodes[node]['features']
        all_nodes_targets[node] = graph.nodes[node][target]

# convert to dataframe
all_nodes_df = pd.DataFrame.from_dict(all_nodes, orient='index')
gc.collect()

edge_df = nx.to_pandas_edgelist(graphs[0], source='source', target='target')
for graph in tqdm(graphs[1:]):
    edge_df = pd.concat([edge_df, nx.to_pandas_edgelist(graph, source='source', target='target')])
  

edge_df.reset_index(drop=True, inplace=True)
graph = StellarGraph(nodes=all_nodes_df, edges=edge_df)

In [ ]:
# some convenience bits

numbers = list(range(df.time.nunique()))
timesteps = dict(zip(list(df.sort_values('time').time.unique()), numbers))
df['node_name'] = df.apply(lambda x: str(x.lat_long)+ '-' + str(timesteps[x.time]), axis=1)
all_nodes_targets = dict(zip(df.node_name, df.NO2))
timesteps_dict = {v: k for k, v in timesteps.items()}

### Train model on graph without test location, then predict on full graph one node at a time (takes a LONG time to run predictions)

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
nodes = list(graph.nodes())
batch_size = 50
num_samples = [3, 5]
layer_sizes = [64, 64]
dropout = 0.5
learning_rate = 0.001 
aggregator = MeanAggregator
epochs = 10
all_predictions = {}
all_targets = {}
predictions = {}
all_targets = {}
predictions_dfs = {}
random.seed(42)
tf.random.set_seed(42)
np.random.seed(42)

test_lat_longs = [(51.427864, -2.563742)]

for lat_long in test_lat_longs:
  test_lat_longs = [lat_long]
  train_lat_longs = [(lat, long) for lat, long in lat_longs if (
      lat, long) not in test_lat_longs]

  test_nodes = test_lat_longs

  test_nodes = [node for node in nodes if ((get_tuple(node) in test_nodes))]
  train_nodes = list(set(nodes) - set(test_nodes))

  # remove all nodes for testing lat longs
  subgraph = graph.subgraph(train_nodes)

  test_labels = {node: all_nodes_targets[node] for node in test_nodes}
  train_labels = {node: all_nodes_targets[node] for node in train_nodes}

  generator = GraphSAGENodeGenerator(subgraph, batch_size, num_samples)

  # create numpy array from values of dict
  train_targets = np.array(list(train_labels.values()))
  test_targets = np.array(list(test_labels.values()))
  train_gen = generator.flow(train_nodes, targets=train_targets)


  graphsage_model = GraphSAGE(
      layer_sizes=layer_sizes, generator=generator, bias=True, dropout=dropout, aggregator=aggregator, kernel_initializer=initializers.glorot_uniform(seed=0)
  )

  x_inp, x_out = graphsage_model.in_out_tensors()
  prediction = layers.Dense(units=1, activation="relu",
                            kernel_initializer=initializers.glorot_uniform(seed=0))(x_out)

  # compile model

  model = Model(inputs=x_inp, outputs=prediction)

  model.compile(
      optimizer=optimizers.Adam(learning_rate=learning_rate),
      loss=losses.mean_squared_error,
      metrics=[metrics.mean_absolute_error,
              metrics.mean_absolute_percentage_error],
  )

# train the model

  history = model.fit(
      train_gen,
      epochs=epochs,
      verbose=1,
      shuffle=False)
  
  test_targets_dict = dict(zip(test_nodes, test_targets))
  

  # add test nodes back in
  new_gen = GraphSAGENodeGenerator(graph, batch_size, num_samples)
  new_graph = graph
  test_all_nodes_df = all_nodes_df.copy()

  for timestep in tqdm(list(timesteps.values())):
      # get the node for that timestep
      new_nodes = [node for node in test_nodes if int(
          node.split("-")[-1]) == timestep]
      if len(new_nodes) == 0:
          continue
      targets = [test_targets_dict[node] for node in new_nodes]
      # get the generator flow for that timestep
      test_gen = new_gen.flow(new_nodes, targets=targets)
      del new_gen
      # predict on that timestep
      test_predictions = model.predict(test_gen, verbose=0).flatten()
      
      # save predictions for all test nodes
      for i, node in enumerate(new_nodes):
          predictions[node] = test_predictions[i]
          # all_targets[node] = test_targets[i]
      # change node attributes in all_nodes_df and make new graph then create new generator
      test_all_nodes_df.loc[new_nodes].iloc[:, -1] = test_predictions
      new_graph = StellarGraph(nodes=test_all_nodes_df, edges=edge_df)
      new_gen = GraphSAGENodeGenerator(new_graph, batch_size, num_samples)
      del new_graph,  test_gen, test_predictions
      if timestep % 500 == 10: # save in case of colab runtime timeout
          pd.DataFrame(list(predictions.items()), columns=['key', 'value']).to_csv(
              'test_predictions.csv', index=False)
      if timestep % 50 == 0:
          K. clear_session() # needed to stop Keras memory leak
          gc.collect()


  target_df = pd.DataFrame.from_dict(all_nodes_targets, orient='index')
  target_df.reset_index(inplace=True)
  target_df.columns = ['key', 'actual']
  pred = pd.DataFrame(list(predictions.items()), columns=['key', 'value'])
  pred['time'] = pred['key'].apply(
      lambda x: timesteps_dict[int(x.split("-")[-1])])
  pred['lat_long'] = pred['key'].apply(get_tuple)
  pred = pd.merge(pred, target_df, how='left', on='key')
  pred.columns = ['node', 'prediction', 'time', 'lat_long', 'actual']
  pred.to_csv('predictions.csv', index=False)




### PLOT

In [ ]:

#  plot predictions vs actuals for each lat_long
fig = nice_plot(go.Figure(), title='Predictions vs actuals for each location',
                x_label='Time', y_label='NO2', width=1000, height=400)
for lat_long in pred['lat_long'].unique():
    fig.add_trace(go.Scatter(x=pred[pred['lat_long'] == lat_long]['time'],
                             y=pred[pred['lat_long']
                                           == lat_long]['actual'],
                             mode='lines', name=f'{lat_long_to_location[lat_long]} actual', opacity=0.4))
    fig.add_trace(go.Scatter(x=pred[pred['lat_long'] == lat_long]['time'],
                             y=pred[pred['lat_long']
                                           == lat_long]['prediction'],
                             mode='lines', name=f'{lat_long_to_location[lat_long]} prediction'))

fig.show()